# Dates and Times the Smart Way


## 1. `datetime` Objects

In [2]:
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd

In [3]:
print(datetime.today())

2025-10-07 18:54:05.281262


In [4]:
type(datetime.today())

datetime.datetime

### Operations on `datetime` objects

A nice feature of `datetime` objects is that they allow us to pretty easily quantify how much time has elapsed between two, known dates or times. For example, have you ever wondered how many days elapsed between the releases of "Red" and "The Life of a Showgirl?" Well, we can use `datetime` objects to figure out:

In [5]:
print(datetime(2025,10,3) - datetime(2012,10,22))

4729 days, 0:00:00


this is a fairly precise answer, but we can get the number of days as a whole number:

In [6]:
print((datetime(2025,10,3) - datetime(2012,10,22)).days)


4729


as a side note, let's look at what kind of object the difference between two dates is:

In [7]:
type(datetime(2025,10,3) - datetime(2012,10,22))

datetime.timedelta

If need be, we can also then use attributes of this `timedelta` object in mathematical operations

In [8]:
nDays = (datetime(2025,10,3) - datetime(2012,10,22)).days

if(nDays % 13 == 0):
    print("TSwifts albums released in even intervals of 13 days?!?!")
else:
    print("Nah")

Nah


Importantly, we can also use the `datetime.strftime()` function to print a `datetime` object to a string in any format we so desire.

In [9]:
myBDay = datetime(2025,10,24)

outtext = f'''
Different ways to print a date:
\t Way 1: {myBDay.strftime('%Y-%m-%d')}
\t Way 2: {myBDay.strftime('%m-%d-%Y')}
\t Way 3: {myBDay.strftime('%m/%d/%Y')}
\t Way 4: {myBDay.strftime('%A %B %d, %Y')}
\t Year only: {myBDay.strftime('%Y')}
\t Month only: {myBDay.strftime('%m')}
\t Day only: {myBDay.strftime('%d')}
'''
print(outtext)


Different ways to print a date:
	 Way 1: 2025-10-24
	 Way 2: 10-24-2025
	 Way 3: 10/24/2025
	 Way 4: Friday October 24, 2025
	 Year only: 2025
	 Month only: 10
	 Day only: 24



### Real Use Case: Landsat File Names

A very common application of dealing with dates in the geosciences is diagnosing the date and time of a satellite data acquisition based on the name of the file. While this information is in the metadata of the file, it is often convenient to be able to do things like determine the length of time between two satellite data acquisitions __before__ downloading the actual file. Additionally, if we want to build a time series of satellite observations – potentially from different, but similar, satellite platforms – we might need to know the actual intervals between satellite overflights. Below is a realworld example of how you might use `datetime` to determine the date of a Landsat acquisition. Below is an image of the Landsat file naming convention. 

Make note of the `datetime.strptime()` function below, which is largely the reverse of the `datetime.strftime()` function – it takes a string with instructions on how that string embeds a date and time and then converts it to a date time object.

![Landsat image naming convention](../img/ls_naming_convention.png)

In [10]:
lsfname = 'LC08_L1GT_029030_20151209_20160131_01_RT'
 
acdate_str = lsfname.split('_')[3] # splits up by underscore and then reads that the 4th object in the vector of information is the date string
acdate_dt = datetime.strptime(acdate_str, '%Y%m%d')
print(acdate_dt)

2015-12-09 00:00:00


### Challenge 1: Can You Determine How Many Days Elapsed Between Landsat Retrievals?

In [11]:
lsfname2 = 'LC08_L1TP_029030_20250915_20250920_02_T1'
acdate_str2 = lsfname2.split('_')[3] # splits up by underscore and then reads that the 4th object in the vector of information is the date string
acdate_dt2 = datetime.strptime(acdate_str2, '%Y%m%d')

retIntervalText = f'''
Date of First Retrieval: {acdate_dt.strftime('%Y-%m-%d')}
Date of Second Retrieval: {acdate_dt2.strftime('%Y-%m-%d')}
Number of days between: {(acdate_dt2 - acdate_dt).days}
'''

print(retIntervalText)



Date of First Retrieval: 2015-12-09
Date of Second Retrieval: 2025-09-15
Number of days between: 3568



## 2. `timedelta` Objects

As we saw above, when we computed the difference between two dates, Python returned a `timedelta` object. We can use the `timedelta` function to add a number of days or times to a `datetime` object. The result is a `datetime` object.

In [12]:
newDate = datetime.today() + timedelta(days=10)
print(newDate.strftime('%Y-%m-%d'))


2025-10-17


### Challenge 2: Number of Days? 

Using `datetime` and `timedelta`, compute how many days will elapse between now and 13 years from now. The __conventional__ answer is 4,380. Why is that the conventional answer and what is the __correct__ answer?   

In [13]:
print(4380/13)
print(12*365)

date1 = datetime.today()
date2 = datetime(2025+13,10,6)
print(date2.strftime('%Y-%m-%d'))
print((date2-date1).days)


336.9230769230769
4380
2038-10-06
4746


Challenge 3: Expected File Names?

The Landsat revisit interval for a location is 16 days (some locations that are at the intersection of images have 8 day revisit intervals). You want a collection of approximately the next 2 years of Landsat images at a 16 day interval, and you want to download the images as soon as they are available. From the landsat image at the file referenced below, what are the expected file names of all of the files?

In [14]:
lsf_collection_file = 'LC08_L1TP_029030_20250915_20250920_02_T1'
termination_date = datetime(2025+2, 9,15)
end_date = termination_date
collection_file_date = lsf_collection_file.split('_')[3]
cfd = datetime.strptime(collection_file_date, '%Y%m%d')
prefix = "LC08_L1TP_029030_"

p_date = lsf_collection_file.split('_')[4]
processing_date = datetime.strptime(p_date, '%Y%m%d')
suffix = "_02_T1"

filenames = []

current = cfd

while current <= end_date:
    filenames.append(f"{prefix}{current.strftime('%Y%m%d')}_{processing_date.strftime('%Y%m%d')}{suffix}")
    current += timedelta(days=16)
    processing_date = current + timedelta(days=5)
#it looks like we need to say that the processing date is 5 days after collection date - no way can an image be processed before it is captured!

print(*filenames, sep = '\n')



LC08_L1TP_029030_20250915_20250920_02_T1
LC08_L1TP_029030_20251001_20251006_02_T1
LC08_L1TP_029030_20251017_20251022_02_T1
LC08_L1TP_029030_20251102_20251107_02_T1
LC08_L1TP_029030_20251118_20251123_02_T1
LC08_L1TP_029030_20251204_20251209_02_T1
LC08_L1TP_029030_20251220_20251225_02_T1
LC08_L1TP_029030_20260105_20260110_02_T1
LC08_L1TP_029030_20260121_20260126_02_T1
LC08_L1TP_029030_20260206_20260211_02_T1
LC08_L1TP_029030_20260222_20260227_02_T1
LC08_L1TP_029030_20260310_20260315_02_T1
LC08_L1TP_029030_20260326_20260331_02_T1
LC08_L1TP_029030_20260411_20260416_02_T1
LC08_L1TP_029030_20260427_20260502_02_T1
LC08_L1TP_029030_20260513_20260518_02_T1
LC08_L1TP_029030_20260529_20260603_02_T1
LC08_L1TP_029030_20260614_20260619_02_T1
LC08_L1TP_029030_20260630_20260705_02_T1
LC08_L1TP_029030_20260716_20260721_02_T1
LC08_L1TP_029030_20260801_20260806_02_T1
LC08_L1TP_029030_20260817_20260822_02_T1
LC08_L1TP_029030_20260902_20260907_02_T1
LC08_L1TP_029030_20260918_20260923_02_T1
LC08_L1TP_029030

## 3. `datetime` reference:

https://docs.python.org/3/library/datetime.html